# **COBEQ 2025**

## Libs

In [2]:
import pandas as pd
import numpy as np
np.random.seed(1)
import optuna
import itertools
import shutil
from functools import partial

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import time

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
import sklearn.metrics as metrics

ponte = pd.read_pickle(r'Data\Data_Ponte_dos_Remedios.pkl')
del ponte['o3']
guarulhos = pd.read_pickle(r'Data\Data_Guarulhos.pkl')
guarulhos = guarulhos[['date','o3']]

data = ponte.merge(guarulhos, on='date', how='outer')
data.reset_index(drop=True)

import joblib
import pickle
from IPython.display import clear_output
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False  # Disable model summary
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

from TimeObjectModule import TimeObject

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## **Experimental Planning**

In [24]:
for h in [14, 30, 60, 90, 120]:
    for k in [1, 2, 3, 4, 5,]:
        x = TimeObject(data, column='pm10', agg_freq='D')
        x.fixed_origin_rolling_window_cross_validation(
            initial_train_date=0,
            total_data_points=1000+(k*h),
            h=h,
            # plot_interval=True,
            width=1100,height=400
        )
        # print()

In [20]:
for horizon in [14, 30, 60, 90, 120]:
    print(f'H = {horizon}')
    max_k_fold = 5
    for k_fold in range(0,max_k_fold):
        obj = TimeObject(df=data, column='pm10', agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df))-(k_fold*horizon),
            h=horizon,
            # plot_interval=True
        )
        print(f'Fold: {k_fold+1} | Train = {len(obj.Y_train)} | Percent = {round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}')
    print('=================')
    for k_fold in range(0,max_k_fold):
        obj = TimeObject(df=data, column='pm10', agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df)-(max_k_fold*horizon))-(k_fold*horizon),
            h=horizon,
            # plot_interval=True
        )
        print(f'Fold: {k_fold+1} | Train = {len(obj.Y_train)} | Percent = {round(100*len(obj.Y_train)/len(obj.nixtla_df),3)}')

    print('\n')

H = 14
Fold: 1 | Train = 2452 | Percent = 99.432
Fold: 2 | Train = 2438 | Percent = 98.865
Fold: 3 | Train = 2424 | Percent = 98.297
Fold: 4 | Train = 2410 | Percent = 97.729
Fold: 5 | Train = 2396 | Percent = 97.161
Fold: 1 | Train = 2382 | Percent = 96.594
Fold: 2 | Train = 2368 | Percent = 96.026
Fold: 3 | Train = 2354 | Percent = 95.458
Fold: 4 | Train = 2340 | Percent = 94.891
Fold: 5 | Train = 2326 | Percent = 94.323


H = 30
Fold: 1 | Train = 2436 | Percent = 98.783
Fold: 2 | Train = 2406 | Percent = 97.567
Fold: 3 | Train = 2376 | Percent = 96.35
Fold: 4 | Train = 2346 | Percent = 95.134
Fold: 5 | Train = 2316 | Percent = 93.917
Fold: 1 | Train = 2286 | Percent = 92.701
Fold: 2 | Train = 2256 | Percent = 91.484
Fold: 3 | Train = 2226 | Percent = 90.268
Fold: 4 | Train = 2196 | Percent = 89.051
Fold: 5 | Train = 2166 | Percent = 87.835


H = 60
Fold: 1 | Train = 2406 | Percent = 97.567
Fold: 2 | Train = 2346 | Percent = 95.134
Fold: 3 | Train = 2286 | Percent = 92.701
Fold: 4 | 

## **Optuna**

### N-HiTS

In [3]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 90, 1100, step=1)
    n_stacks = trial.suggest_int('n_stacks', 3, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 7, step=1)
    max_steps = trial.suggest_int('max_steps', 10, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    n_pool_kernel_size = trial.suggest_categorical('n_pool_kernel_size', [list(combination) for combination in list(itertools.product([1, 2, 3], repeat=3))])
    n_freq_downsample = trial.suggest_categorical('n_freq_downsample', [list(combination) for combination in list(itertools.product([1, 4, 12, 52], repeat=3))])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    max_k_fold = 5
    for k_fold in range(0,max_k_fold):
        print(f'\nPollutant = {pollutant} \nh = {horizon} \nTrial = {trial.number+1}\nFold = {k_fold+1}\n')
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data, column=pollutant, agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df))-(k_fold*horizon),
            h=horizon,
            plot_interval=False
        )

        # Define the model
        model = NHITS(
            h=horizon,
            input_size=input_size,
            stack_types=n_stacks*['identity'],
            n_freq_downsample=n_freq_downsample+(n_stacks-len(n_freq_downsample))*[1],
            n_blocks=n_stacks*[n_blocks],
            n_pool_kernel_size=(n_stacks-len(n_pool_kernel_size))*[1]+n_pool_kernel_size,
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=max_steps,
            val_check_steps=10,
            early_stop_patience_steps=int(np.round(max_steps/(20),0)),
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False, val_size=horizon+1)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NHITS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
        
        clear_output(wait=True)

    try:
        directory_path = "lightning_logs"
        if os.path.exists(directory_path):
            shutil.rmtree(directory_path)
    except:
        ...

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'poll': pollutant,
        'freq': 'D',
        'fold': max_k_fold,
        'train_len': f'{(len(obj.nixtla_df))-(max_k_fold*horizon)} - {(len(obj.nixtla_df))-(0*horizon)}',
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'n_pool_kernel_size': n_pool_kernel_size,
        'n_freq_downsample': n_freq_downsample,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape, mae  # Any metric you want to optimize

for pollutant in data[['pm10']]:
    for h in [90]:
        # Initialize the results list
        results = []
        # Define the optimization study_nhits
        study_nhits = optuna.create_study(directions=['minimize','minimize'])  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nhits.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=200)

        clear_output(wait=True)
        NHITS_W = pd.DataFrame(results)

        output_dir = fr'Results COBEQ\NHITS (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        NHITS_W.to_pickle(fr'Results COBEQ\NHITS (D)\{pollutant}\{h}D_Df.pkl')
        joblib.dump(study_nhits, fr"Results COBEQ\NHITS (D)\{pollutant}\{h}D_Study.pkl")

[I 2025-04-12 15:17:37,389] Trial 199 finished with values: [39.045902, 9.382224] and parameters: {'input_size': 633, 'n_stacks': 3, 'n_blocks': 7, 'max_steps': 87, 'local_scalar_type': 'minmax', 'n_pool_kernel_size': [3, 3, 3], 'n_freq_downsample': [1, 1, 1]}.


### NBEATS

In [15]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 90, 1100, step=1)
    n_stacks = trial.suggest_int('n_stacks', 2, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 10, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    interpretability = trial.suggest_categorical('interpretability', [list(combination) for combination in list(itertools.product(['seasonality', 'trend', 'identity'], repeat=2))])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    max_k_fold = 5
    for k_fold in range(0,max_k_fold):
        print(f'\nPollutant = {pollutant} \nh = {horizon} \nTrial = {trial.number+1}\nFold = {k_fold+1}\n')
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data, column=pollutant, agg_freq='D')
        obj.fixed_origin_rolling_window_cross_validation(
            initial_train_date=None,
            total_data_points=(len(obj.nixtla_df))-(k_fold*horizon),
            h=horizon,
            plot_interval=False
        )

        # Define the model
        model = NBEATS(
            h=horizon,
            input_size=input_size,
            stack_types=interpretability+(n_stacks-len(interpretability))*['identity'],
            n_blocks=n_stacks * [n_blocks],
            max_steps=max_steps,
            learning_rate=1e-3,
            val_check_steps=10,
            early_stop_patience_steps=int(np.round(max_steps/(20),0)),
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False, val_size=horizon+1)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NBEATS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
            
        clear_output(wait=True)

        try:
            directory_path = "lightning_logs"
            if os.path.exists(directory_path):
                shutil.rmtree(directory_path)
        except:
            ...

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'pollutant': pollutant,
        'freq': 'D',
        'fold': max_k_fold,
        'train_len': f'{(len(obj.nixtla_df))-(max_k_fold*horizon)} - {(len(obj.nixtla_df))-(0*horizon)}',
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'interpretability': interpretability,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape, mae  # Any metric you want to optimize

for pollutant in data[['pm10']]:
    for h in [90]:
        # Initialize the results list
        results = []
        # Define the optimization study_nbeats
        study_nbeats = optuna.create_study(directions=['minimize','minimize'])  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nbeats.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=200)

        clear_output(wait=True)
        NBEATS_W = pd.DataFrame(results)

        output_dir = fr'Results COBEQ\NBEATS (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        NBEATS_W.to_pickle(fr'Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Df.pkl')
        joblib.dump(study_nbeats, fr"Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Study.pkl")

[I 2025-04-13 09:47:10,279] Trial 199 finished with values: [39.852276, 9.647514000000001] and parameters: {'input_size': 925, 'n_stacks': 2, 'n_blocks': 2, 'max_steps': 494, 'local_scalar_type': 'boxcox', 'interpretability': ['trend', 'identity']}.


## **Statistical**

In [17]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoCES, AutoETS, AutoTheta

for pollutant in ['pm10',]:
    for h in [14, 30, 60, 90]: #  30, 60, 90, 120
        results_stats = pd.DataFrame()

        mape = []
        smape = []
        max = []
        mae = []
        mse = []

        max_k_fold = 5
        for k_fold in range(0,max_k_fold):
            print(f'\nPollutant = {pollutant} \nh = {h}\nFold = {k_fold+1}\n')
            # Instantiate TimeObject and prepare training data
            obj = TimeObject(df=data, column=pollutant, agg_freq='D')
            obj.fixed_origin_rolling_window_cross_validation(
                initial_train_date=None,
                total_data_points=(len(obj.nixtla_df))-(k_fold*h),
                h=h,
                plot_interval=False
            )

            season_length = 365 # Monthly data 

            models = [
                # AutoARIMA(season_length=season_length, alias='AutoARIMA'),
                AutoCES(season_length=season_length, model='Z', alias='AutoCES-Z'),
                AutoCES(season_length=season_length, model='S', alias='AutoCES-S'),
                AutoCES(season_length=season_length, model='P', alias='AutoCES-P'),
                AutoCES(season_length=season_length, model='N', alias='AutoCES-N'),
                AutoTheta(season_length=season_length, decomposition_type="multiplicative", alias='AutoTheta-Multi'),
                AutoTheta(season_length=season_length, decomposition_type="additive", alias='AutoTheta-Add'),
            ]
            models = models + [
                AutoETS(season_length=season_length, model=ets, alias=f'AutoETS-{ets}')
                for ets in ['ZAZ', 'ZAN', 'ZAA', 'ZAM', 'ZNN']]

            frct = StatsForecast(models=models, freq='D')
            frct.fit(df=obj.Y_train)
            predicted = frct.predict(h=h)

            columns = predicted.columns
            columns = columns[(columns != 'ds') & (columns != 'unique_id')]

            for method in columns:
                obj.metrics_(predicted, method=method)
                results_stats = pd.concat([results_stats, pd.DataFrame({
                    'pollutant': [pollutant],
                    'method': [method],
                    'freq': ['D'],
                    'h': [h],
                    'mape': [obj.metrics['mape']],
                    'smape': [obj.metrics['smape']],
                    'max': [obj.metrics['max']],
                    'mae': [obj.metrics['mae']],
                    'mse': [obj.metrics['mse']]
                })])
            
            # ======================================================================================================

            nbeats = joblib.load(fr"Results COBEQ\NBEATS (D)\{pollutant}\{h}D_Study.pkl")
            model = NBEATS(
                h=h,
                input_size=nbeats.best_trials[0].params.get('input_size'),
                stack_types=nbeats.best_trials[0].params.get('interpretability')+(nbeats.best_trials[0].params.get('n_stacks')-len(nbeats.best_trials[0].params.get('interpretability')))*['identity'],
                n_blocks=nbeats.best_trials[0].params.get('n_stacks') * [nbeats.best_trials[0].params.get('n_blocks')],
                max_steps=nbeats.best_trials[0].params.get('max_steps'),
                learning_rate=1e-3,
                val_check_steps=10,
            )
            fcst = NeuralForecast(
                models=[model],
                freq='D',
                local_scaler_type=nbeats.best_trials[0].params.get('local_scalar_type')
            )
            fcst.fit(df=obj.Y_train, verbose=False)
            predicted = fcst.predict(df=obj.Y_train, verbose=False)
            obj.metrics_(predicted, method='NBEATS')
            results_stats = pd.concat([results_stats, pd.DataFrame({
                'pollutant': [pollutant],
                'method': ['NBEATS'],
                'freq': ['D'],
                'h': [h],
                'mape': [obj.metrics['mape']],
                'smape': [obj.metrics['smape']],
                'max': [obj.metrics['max']],
                'mae': [obj.metrics['mae']],
                'mse': [obj.metrics['mse']]
            })])

            # ======================================================================================================
            
            nhits = joblib.load(fr"Results COBEQ\NHITS (D)\{pollutant}\{h}D_Study.pkl")
            model = NHITS(
                h=h,
                input_size=nhits.best_trials[0].params.get('input_size'),
                stack_types=nhits.best_trials[0].params.get('n_stacks')*['identity'],
                n_freq_downsample=nhits.best_trials[0].params.get('n_freq_downsample')+(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_freq_downsample')))*[1],
                n_blocks=nhits.best_trials[0].params.get('n_stacks')*[nhits.best_trials[0].params.get('n_blocks')],
                n_pool_kernel_size=(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_pool_kernel_size')))*[1]+nhits.best_trials[0].params.get('n_pool_kernel_size'),
                pooling_mode="MaxPool1d",
                activation="ReLU",
                interpolation_mode='linear',
                max_steps=nhits.best_trials[0].params.get('max_steps'),
                val_check_steps=10,
            )
            fcst = NeuralForecast(
                models=[model],
                freq='D',
                local_scaler_type=nhits.best_trials[0].params.get('local_scalar_type')
            )
            fcst.fit(df=obj.Y_train, verbose=False)
            predicted = fcst.predict(df=obj.Y_train, verbose=False)
            obj.metrics_(predicted, method='NHITS')
            results_stats = pd.concat([results_stats, pd.DataFrame({
                'pollutant': [pollutant],
                'method': ['NHITS'],
                'freq': ['D'],
                'h': [h],
                'mape': [obj.metrics['mape']],
                'smape': [obj.metrics['smape']],
                'max': [obj.metrics['max']],
                'mae': [obj.metrics['mae']],
                'mse': [obj.metrics['mse']]
            })])
            
            clear_output(wait=True)

        # ======================================================================================================

        results_stats = pd.DataFrame(results_stats)

        output_dir = fr'Results COBEQ\Stats (D)\{pollutant}'
        os.makedirs(output_dir, exist_ok=True)
        results_stats.to_pickle(fr'Results COBEQ\Stats (D)\{pollutant}\{h}D_Df.pkl')


Pollutant = pm10 
h = 90
Fold = 5



Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 10.9 M | train
-------------------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.568    Total estimated model params size (MB)
76        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=87` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [36]:
results_stats = joblib.load(r'Results COBEQ\Stats (D)\pm10\60D_Df.pkl')

In [37]:
results = results_stats.groupby(['pollutant','method','h','freq'])[['mape', 'smape', 'max', 'mae', 'mse']].mean().reset_index().sort_values('smape')
results

,pollutant,method,h,freq,mape,smape,max,mae,mse
11,pm10,NBEATS,60,D,54.394558,39.422596,35.455774,10.302232,174.203814
12,pm10,NHITS,60,D,59.248128,40.221806,35.984348,10.916768,194.927824
2,pm10,AutoCES-S,60,D,61.747942,41.334518,34.418710,10.993020,190.824902
4,pm10,AutoETS-ZAA,60,D,60.947488,41.341806,33.610480,11.050168,198.340240
5,pm10,AutoETS-ZAM,60,D,63.178216,41.535840,33.350936,11.163496,201.290944
3,pm10,AutoCES-Z,60,D,64.523390,41.644108,35.837752,11.297492,210.149334
8,pm10,AutoETS-ZNN,60,D,88.141228,49.440976,36.923542,15.012138,378.710626
10,pm10,AutoTheta-Multi,60,D,111.636506,65.267502,51.513694,20.540366,700.368024
9,pm10,AutoTheta-Add,60,D,115.211282,69.790644,46.403070,21.121166,695.747784
0,pm10,AutoCES-N,60,D,65.764796,95.098624,49.728050,17.809154,496.847390


In [ ]:
results_stats = joblib.load(r'Results COBEQ\NHITS (D)\pm10\60D_Study.pkl')
optuna.plot_contour()

AttributeError: module 'optuna' has no attribute 'plot_contour'

# **ANALYSIS**

In [ ]:
df = pd.DataFrame()
for model in ['NHITS','NBEATS']:
    for H in [12,26,52,78]:
        x = joblib.load(fr'Results COBEQ/Stats (W)/pm10/{H}W_Df.pkl').sort_values(['smape','mae']).reset_index()
        metrics = ['smape', 'mae', 'mape', 'mse', 'max']
        for metric in metrics:
            x[f'{metric}_rank'] = x[metric].rank(method='min')

        # If you want best = 1, worst = 21, and ranks to be integers
        x[[f'{m}_rank' for m in metrics]] = x[[f'{m}_rank' for m in metrics]].astype(int)
        df = pd.concat([df, x[x['method'] == model]])
        del df['index']
df.to_excel(r'x.xlsx')

In [ ]:
df = pd.DataFrame()
for model in ['NHITS','NBEATS']:
    for H in [12,26,52,78]:
        x = joblib.load(fr'C:\Users\gustavo.filho\Documents\Python\GitHub\TCC\Results COBEQ\{model} (W)\pm10\{H}W_Df.pkl').sort_values(['smape','mae']).reset_index()
        df = pd.concat([df, pd.DataFrame(x.iloc[0,:]).T])

df.to_excel(r'x.xlsx')

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoCES, AutoETS, AutoTheta

results_stats = []
for pollutant in ['pm10']:
    for h in [365]:

        obj = TimeObject(data, pollutant, agg_freq='D')
        obj.NIXTLA_train_test(split=h)

        season_length = 365 # Monthly data 
        horizon = len(obj.Y_train) # number of predictions

        models = [
            # AutoARIMA(season_length=season_length, alias='AutoARIMA'),
            AutoCES(season_length=season_length, model='Z', alias='AutoCES-Z'),
            AutoCES(season_length=season_length, model='S', alias='AutoCES-S'),
            AutoCES(season_length=season_length, model='P', alias='AutoCES-P'),
            AutoCES(season_length=season_length, model='N', alias='AutoCES-N'),
            AutoTheta(season_length=season_length, decomposition_type="multiplicative", alias='AutoTheta-Multi'),
            AutoTheta(season_length=season_length, decomposition_type="additive", alias='AutoTheta-Add'),
        ]
        models = models + [
            AutoETS(season_length=season_length, model=ets, alias=f'AutoETS-{ets}')
            for ets in [f"{e}{t}{s}" for e in ['Z',]
                        for t in ['Z', 'A', 'N'] 
                        for s in ['Z', 'A', 'M', 'N'] 
        ]]

        frct = StatsForecast(models=models, freq='D')
        frct.fit(df=obj.Y_train)
        predicted = frct.predict(h=h)

        columns = predicted.columns
        columns = columns[(columns != 'ds') & (columns != 'unique_id')]

        results_stats = pd.DataFrame()
        for method in columns:
            obj.plot_forecast(predicted, show_metrics=True, method=method)

In [42]:
for pollutant in ['pm10']:
    for h in [14, 30, 60, 90]:

        obj = TimeObject(data, pollutant, agg_freq='D')
        obj.NIXTLA_train_test(split=h)
        horizon = len(obj.Y_train) # number of predictions

        # ======================================================================================================

        nbeats = joblib.load(fr"Results COBEQ\NBEATS (W)\{pollutant}\78W_Study.pkl")
        model = NBEATS(
            h=h,
            input_size=nbeats.best_trials[0].params.get('input_size'),
            stack_types=nbeats.best_trials[0].params.get('interpretability')+(nbeats.best_trials[0].params.get('n_stacks')-len(nbeats.best_trials[0].params.get('interpretability')))*['identity'],
            n_blocks=nbeats.best_trials[0].params.get('n_stacks') * [nbeats.best_trials[0].params.get('n_blocks')],
            max_steps=nbeats.best_trials[0].params.get('max_steps'),
            learning_rate=1e-3,
            val_check_steps=10,
        )
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=nbeats.best_trials[0].params.get('local_scalar_type')
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        predicted = fcst.predict(df=obj.Y_train, verbose=False)
        obj.plot_forecast(predicted, method='NBEATS', show_metrics=True)

        # ======================================================================================================
        
        nhits = joblib.load(fr"Results COBEQ\NHITS (W)\{pollutant}\78W_Study.pkl")
        model = NHITS(
            h=h,
            input_size=nhits.best_trials[0].params.get('input_size'),
            stack_types=nhits.best_trials[0].params.get('n_stacks')*['identity'],
            n_freq_downsample=nhits.best_trials[0].params.get('n_freq_downsample')+(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_freq_downsample')))*[1],
            n_blocks=nhits.best_trials[0].params.get('n_stacks')*[nhits.best_trials[0].params.get('n_blocks')],
            n_pool_kernel_size=(nhits.best_trials[0].params.get('n_stacks')-len(nhits.best_trials[0].params.get('n_pool_kernel_size')))*[1]+nhits.best_trials[0].params.get('n_pool_kernel_size'),
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=nhits.best_trials[0].params.get('max_steps'),
            val_check_steps=10,
        )
        fcst = NeuralForecast(
            models=[model],
            freq='D',
            local_scaler_type=nhits.best_trials[0].params.get('local_scalar_type')
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        predicted = fcst.predict(df=obj.Y_train, verbose=False)
        obj.plot_forecast(predicted, show_metrics=True)

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 7.7 M  | train
-------------------------------------------------------
7.7 M     Trainable params
783       Non-trainable params
7.7 M     Total params
30.842    Total estimated model params size (MB)
85        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=167` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

mae: 6.71694
mape: 46.35123
mse: 67.92055
max: 20.37445
smape: 33.18373
